In [1]:
# ------------------------------------------define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.Tensorflow_helper import choose_gpu_by_id
# ------------------------------------------define GPU id/s to use
GPU_IDS = '0,1'
GPUS = choose_gpu_by_id(GPU_IDS)
print(GPUS)
# ------------------------------------------jupyter magic config
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# ------------------------------------------ import helpers
# this should import glob, os, and many other standard libs
# local imports
from src.utils.Notebook_imports import *
from src.utils.Utils_io import Console_and_file_logger, init_config

# import external libs
from tensorflow.python.client import device_lib
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import cv2
import pandas as pd

EXPERIMENT = 'cv_baseline_finalconv5/8_64_64__8_3_3_tenc64_conv1_MSE_NOnorm_augshiftscalerotbrigthness_taug3_3_batch8'
#EXPERIMENT = 'baseline_label_transpose_smooth05/36_5_BiLSTM32_NoBn_conv5_size1_CCE_NOphaseaug_NOaug_b8'
# EXPERIMENT = 'mased_scores/36_5_BiLSTM32_NoBn_conv5_size1_CCE_NOphaseaug_shift_rotate_reflectbordersgridaug'
timestemp = str(datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")) # ad a timestep to each project to make repeated experiments unique

EXPERIMENTS_ROOT = 'exp/'
EXP_PATH = os.path.join(EXPERIMENTS_ROOT, EXPERIMENT, timestemp)
MODEL_PATH = os.path.join(EXP_PATH, 'model', )
TENSORBOARD_PATH = os.path.join(EXP_PATH, 'tensorboard_logs')
CONFIG_PATH = os.path.join(EXP_PATH,'config')
HISTORY_PATH = os.path.join(EXP_PATH, 'history')
ensure_dir(MODEL_PATH)
ensure_dir(TENSORBOARD_PATH)
ensure_dir(CONFIG_PATH)
ensure_dir(HISTORY_PATH)

# define the data paths and fold 
# first to the 4D Nrrd files, 
# second to a dataframe with a mapping of the Fold-number
# Finally the path to the metadata
DATA_PATH_SAX = '/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAX/'
DF_FOLDS = '/mnt/ssd/data/gcn/02_imported_4D_unfiltered/df_kfold.csv'
DF_META = '/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAx_3D_dicomTags_phase'
FOLD = 0

# General params
SEED = 42 # define a seed for the generator shuffle
BATCHSIZE = 8 # 32, 64, 24, 16, 1 for 3D use: 4
GENERATOR_WORKER = BATCHSIZE # if not set, use batchsize
EPOCHS = 100

DIM = [8, 64, 64] # network input shape for spacing of 3, (z,y,x)
T_SHAPE = 36
SPACING = [8, 3, 3] # if resample, resample to this spacing, (z,y,x)

# Model params
DEPTH = 4 # depth of the encoder
FILTERS = 32 # initial number of filters, will be doubled after each downsampling block
M_POOL = [1, 2, 2]# size of max-pooling used for downsampling and upsampling
F_SIZE = [3, 3, 3] # conv filter size
BN_FIRST = False # decide if batch normalisation between conv and activation or afterwards
BATCH_NORMALISATION = True # apply BN or not
PAD = 'same' # padding strategy of the conv layers
KERNEL_INIT = 'he_normal' # conv weight initialisation
OPTIMIZER = 'adam' # Adam, Adagrad, RMSprop, Adadelta,  # https://keras.io/optimizers/
ACTIVATION = 'relu' # tf.keras.layers.LeakyReLU(), relu or any other non linear activation function
LEARNING_RATE = 1e-4 # start with a huge lr to converge fast
REDUCE_LR_ON_PLAEAU_PATIENCE = 5
DECAY_FACTOR = 0.7 # Define a learning rate decay for the ReduceLROnPlateau callback
MIN_LR = 1e-12 # minimal lr, smaller lr does not improve the model
DROPOUT_min = 0.3 # lower dropout at the shallow layers
DROPOUT_max = 0.5 # higher dropout at the deep layers

# Callback params
MONITOR_FUNCTION = 'loss'
MONITOR_MODE = 'min'
SAVE_MODEL_FUNCTION = 'loss'
SAVE_MODEL_MODE = 'min'
MODEL_PATIENCE = 20
SAVE_LEARNING_PROGRESS_AS_TF = True

# Generator and Augmentation params
BORDER_MODE = cv2.BORDER_REFLECT_101 # border mode for the data generation
IMG_INTERPOLATION = cv2.INTER_LINEAR # image interpolation in the genarator
MSK_INTERPOLATION = cv2.INTER_NEAREST # mask interpolation in the generator
AUGMENT = True # a compose of 2D augmentation (grid distortion, 90degree rotation, brightness and shift)
AUGMENT_PROB = 0.8
AUGMENT_PHASES = True
AUGMENT_PHASES_RANGE = (-3,3)
REPEAT_ONEHOT = True
SHUFFLE = True
RESAMPLE = True
SCALER = 'MinMax' # MinMax, Standard or Robust
# We define 5 target phases and a background phase for the pad/empty volumes 
PHASES = len(['ED#', 'MS#', 'ES#', 'PF#', 'MD#']) # skipped 'pad backround manually added', due to repeating
TARGET_SMOOTHING = True
SMOOTHING_KERNEL_SIZE = 12
SMOOTHING_LOWER_BORDER = 1
SMOOTHING_UPPER_BORDER = 5
SMOOTHING_WEIGHT_CORRECT = 10
config = init_config(config=locals(), save=True)


search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/dynamic-cmr-models
['/gpu:0', '/gpu:1']


In [2]:
from src.models.train_model import train_fold

folds = [i for i in range(4)]

for f in folds:
    info('starting fold: {}'.format(f))
    config_ = config.copy()
    config_['FOLD'] = f
    train_fold(config_)

DEBUG:root:Creating directory /mnt/ssd/git/dynamic-cmr-models/logs/cv_baseline_finalconv5
2021-03-02 22:25:11,688 INFO -------------------- Start --------------------
2021-03-02 22:25:11,689 INFO Working directory: /mnt/ssd/git/dynamic-cmr-models.
2021-03-02 22:25:11,689 INFO Log file: ./logs/cv_baseline_finalconv5/8_64_64__8_3_3_tenc64_conv1_MSE_NOnorm_augshiftscalerotbrigthness_taug3_3_batch8_f0.log
2021-03-02 22:25:11,689 INFO Log level for console: INFO
2021-03-02 22:25:11,690 INFO Is built with tensorflow: True
2021-03-02 22:25:11,747 INFO Visible devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2021-03-02 

['/gpu:0', '/gpu:1']
{'GPU_IDS': '0,1', 'GPUS': ['/gpu:0', '/gpu:1'], 'SEED': 42, 'EXPERIMENT': 'cv_baseline_finalconv5/8_64_64__8_3_3_tenc64_conv1_MSE_NOnorm_augshiftscalerotbrigthness_taug3_3_batch8_f0', 'EXPERIMENTS_ROOT': 'exp/', 'EXP_PATH': 'exp/cv_baseline_finalconv5/8_64_64__8_3_3_tenc64_conv1_MSE_NOnorm_augshiftscalerotbrigthness_taug3_3_batch8_f0/2021-03-02_22_25', 'MODEL_PATH': 'exp/cv_baseline_finalconv5/8_64_64__8_3_3_tenc64_conv1_MSE_NOnorm_augshiftscalerotbrigthness_taug3_3_batch8_f0/2021-03-02_22_25/model', 'TENSORBOARD_PATH': 'exp/cv_baseline_finalconv5/8_64_64__8_3_3_tenc64_conv1_MSE_NOnorm_augshiftscalerotbrigthness_taug3_3_batch8_f0/2021-03-02_22_25/tensorboard_logs', 'CONFIG_PATH': 'exp/cv_baseline_finalconv5/8_64_64__8_3_3_tenc64_conv1_MSE_NOnorm_augshiftscalerotbrigthness_taug3_3_batch8_f0/2021-03-02_22_25/config', 'HISTORY_PATH': 'exp/cv_baseline_finalconv5/8_64_64__8_3_3_tenc64_conv1_MSE_NOnorm_augshiftscalerotbrigthness_taug3_3_batch8_f0/2021-03-02_22_25/histor

2021-03-02 22:25:12,438 INFO no files found, try to load with clean.nrrd/mask.nrrd pattern
2021-03-02 22:25:12,445 INFO Found 278 images/masks in /mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAX/
2021-03-02 22:25:12,445 INFO Patients train: 209
2021-03-02 22:25:12,456 INFO Selected 209 of 278 files with 209 of 279 patients for training fold 0
2021-03-02 22:25:12,456 INFO SAX train CMR: 209, SAX train masks: 209
2021-03-02 22:25:12,456 INFO SAX val CMR: 69, SAX val masks: 69
2021-03-02 22:25:12,469 INFO Check if we find the patient ID and phase mapping for all: 278 files.
2021-03-02 22:25:12,742 INFO Done!
2021-03-02 22:25:12,743 INFO Create DataGenerator
2021-03-02 22:25:12,744 INFO Datagenerator created with: 
 shape: [8, 64, 64]
 spacing: [8, 3, 3]
 batchsize: 8
 Scaler: MinMax
 Images: 209 
 Augment: True 
 Thread workers: 8
2021-03-02 22:25:12,744 INFO Data will be augmented (shift,scale and rotate) with albumentation
2021-03-02 22:25:12,757 INFO Smoothing kernel: 
[ 1.   1.8  2.6  

Shape after the temporal encoder
(None, 36, 8, 4, 4, 512)
Shape after GAP
(None, 36, 512)
Shape after Bi-LSTM layer
(None, 36, 512)
Shape after final conv layer
(None, 36, 5)


2021-03-02 22:25:17,331 INFO feed 4 Tensorboard is ready


...


Epoch 00098: loss did not improve from 0.20517
26/26 [==============================] - 36s 1s/step - loss: 0.2177 - mse_wrapper: 0.2177 - ca_wrapper: 0.8650 - meandiff: 1.6298 - val_loss: 1.0638 - val_mse_wrapper: 1.0638 - val_ca_wrapper: 0.7530 - val_meandiff: 4.0000 - lr: 7.0000e-05
Epoch 99/100
26/26 [==============================] - ETA: 0s - loss: 0.2107 - mse_wrapper: 0.2107 - ca_wrapper: 0.8628 - meandiff: 1.4038
Epoch 00099: loss did not improve from 0.20517
26/26 [==============================] - 34s 1s/step - loss: 0.2107 - mse_wrapper: 0.2107 - ca_wrapper: 0.8628 - meandiff: 1.4038 - val_loss: 0.7109 - val_mse_wrapper: 0.7109 - val_ca_wrapper: 0.8090 - val_meandiff: 4.0625 - lr: 7.0000e-05
Epoch 100/100
26/26 [==============================] - ETA: 0s - loss: 0.2083 - mse_wrapper: 0.2083 - ca_wrapper: 0.8657 - meandiff: 1.5529

2021-03-03 02:31:59,243 INFO (4, 2, 36, 5)
2021-03-03 02:31:59,778 INFO (4, 2, 36, 5)



Epoch 00100: loss did not improve from 0.20517
26/26 [==============================] - 35s 1s/step - loss: 0.2083 - mse_wrapper: 0.2083 - ca_wrapper: 0.8657 - meandiff: 1.5529 - val_loss: 0.8220 - val_mse_wrapper: 0.8220 - val_ca_wrapper: 0.8051 - val_meandiff: 4.1875 - lr: 7.0000e-05


2021-03-03 02:32:02,956 INFO Fold 3 finished after 3734.892 sec


True